# Mnist classification with Tensorflow NN

Handwritten numbers in 28x28=784 pixels matrix ranging from 0-255 into a flattened 784x1 input vector

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Download/load the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
# Test-train split
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = 0.1 * mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [4]:
# Features scaling
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [5]:
# Suffling data: ordered data would be bad while batching
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [6]:
# Batching
BATCH_SIZE=100
train_data = train_data.batch(BATCH_SIZE)
# we dont need batches for validation and test since we dont backpropagate on these sets
# still the model need data of same format so we use one big batch containing all set
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

In [7]:
# NN Model
validation_inputs, validation_targets = next(iter(validation_data)) # make inputs and targets iterable and load first batch
input_size = 784 # 28x28=784 pixels
output_size = 10 # 10 possible digits (0-9)
hidden_layer_size = 200
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)), # transforms tensor to vector
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # first hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'), # second hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), # third hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # last layer, apply classification activation function
])
# use Adam optimizer
# use Sparse categorical crossentropy since it fits classification problems and also applies one-hot encoding
# also ask for accuracy metric
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Training the model
NUM_EPOCHS = 5
model.fit(
    train_data,
    epochs=NUM_EPOCHS,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)], # setup early stopping mechanism
    validation_data=(validation_inputs, validation_targets),
    verbose=2
)

Epoch 1/5
540/540 - 1s - loss: 0.0156 - accuracy: 0.9950 - val_loss: 0.0251 - val_accuracy: 0.9922 - 1s/epoch - 2ms/step
Epoch 2/5
540/540 - 1s - loss: 0.0121 - accuracy: 0.9961 - val_loss: 0.0156 - val_accuracy: 0.9955 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.0104 - accuracy: 0.9966 - val_loss: 0.0105 - val_accuracy: 0.9967 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0088 - accuracy: 0.9968 - val_loss: 0.0170 - val_accuracy: 0.9938 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0115 - accuracy: 0.9961 - val_loss: 0.0144 - val_accuracy: 0.9958 - 1s/epoch - 2ms/step


In [14]:
# Test the model
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 0s 2ms/step - loss: 0.0907 - accuracy: 0.9798
